# Подготовка и изучение данных

In [2]:
import pandas as pd

# Загрузка данных
anime = pd.read_csv('text.csv')

# Функция для фильтрации символов
allowed_chars = set(list("АБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯабвгдеёжзийклмнопрстуфхцчшщъыьэюя0123456789 "))
mapping = lambda s: ''.join(list(filter(lambda c: c in allowed_chars, s)))

# Применение функции к столбцу с текстом песни
anime['description'] = anime['description'].apply(lambda s: mapping(str(s)))

In [3]:
anime.head()

,name,score,aired_on,description,genres
0,Ковбой Бибоп,8.75,1998-04-03,2071 год Человечество колонизировало всю Солне...,"['Экшен', 'Фантастика', 'Космос']"
1,Легенда о новой Белоснежке Притиар,7.16,2001-04-04,Химэно Аваюки была обычной шестнадцатилетней...,"['Драма', 'Фэнтези', 'Романтика', 'Супер сила'..."
2,Космический пират капитан Харлок,7.70,1978-03-14,2977 год Человечество перестало развиваться Вс...,"['Сэйнэн', 'Экшен', 'Приключения', 'Драма', 'Ф..."
3,История из эпохи Сёва,6.84,2011-04-04,Драматическая история семьи Ямадзаки в 39м год...,"['Драма', 'Исторический']"
4,Югио! Зексал,6.42,2011-04-11,История разворачивается в далёком будущем где ...,"['Экшен', 'Фэнтези', 'Сёнен']"


In [4]:
import json
json.loads(anime['genres'][0].replace("'", '"'))

['Экшен', 'Фантастика', 'Космос']

In [4]:
count_genres = {}
for row in anime['genres']:
    r_g = json.loads(row.replace("'", '"'))
    for g in r_g:
        if g not in count_genres:
            count_genres[g] = 0
        count_genres[g] += 1
print(len(count_genres))
print(sorted(count_genres.items(), key=lambda x: x[1]))

40
[('Эротика', 1), ('Машины', 7), ('Безумие', 19), ('Игры', 21), ('Демоны', 22), ('Полиция', 35), ('Гурман', 44), ('Вампиры', 59), ('Самураи', 63), ('Дзёсей', 68), ('Детское', 71), ('Работа', 75), ('Боевые искусства', 77), ('Триллер', 82), ('Космос', 89), ('Пародия', 91), ('Ужасы', 112), ('Музыка', 135), ('Психологическое', 143), ('Сёдзё', 152), ('Военное', 152), ('Спорт', 163), ('Супер сила', 181), ('Меха', 186), ('Гарем', 209), ('Исторический', 213), ('Повседневность', 250), ('Этти', 266), ('Детектив', 277), ('Сэйнэн', 349), ('Сверхъестественное', 381), ('Фантастика', 520), ('Приключения', 568), ('Сёнен', 612), ('Драма', 640), ('Романтика', 657), ('Школа', 697), ('Фэнтези', 864), ('Экшен', 1081), ('Комедия', 1221)]


# Морфологический анализ

In [5]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mansu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mansu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
import pymorphy3

morph = pymorphy3.MorphAnalyzer(lang='ru')
morph_analyze = lambda s: ' '.join([morph.parse(word)[0].normal_form for word in s.split()])

morph_analyze('Красивый и красивая были')
morphed_desc = anime['description'].apply(morph_analyze)

# Подсчёт 5% наиболее частых слов

In [7]:
from collections import Counter

words = ' '.join(morphed_desc).split(' ')
counts = Counter(words)
limit = int(0.05 * len(counts.keys()))
ordered = counts.most_common()
most_common = ordered[:limit]
print(most_common)

[('и', 11272), ('в', 10090), ('он', 4701), ('с', 4198), ('на', 4033), ('не', 3900), ('который', 2973), ('они', 2820), ('что', 2813), ('свой', 2529), ('она', 2036), ('но', 1891), ('это', 1840), ('по', 1634), ('как', 1610), ('быть', 1602), ('а', 1598), ('из', 1592), ('тот', 1475), ('к', 1382), ('мир', 1367), ('всё', 1317), ('о', 1267), ('человек', 1235), ('жизнь', 1191), ('этот', 1173), ('от', 1083), ('её', 1082), ('девушка', 1079), ('за', 1064), ('для', 1034), ('весь', 1014), ('один', 1000), ('новый', 982), ('друг', 949), ('стать', 939), ('школа', 922), ('так', 882), ('же', 882), ('чтобы', 881), ('время', 870), ('год', 861), ('себя', 836), ('только', 802), ('у', 749), ('однако', 725), ('мочь', 723), ('история', 654), ('герой', 628), ('когда', 619), ('сам', 602), ('сила', 588), ('после', 566), ('ещё', 564), ('имя', 551), ('такой', 543), ('другой', 534), ('вместе', 514), ('то', 509), ('день', 504), ('жить', 488), ('ли', 482), ('где', 474), ('теперь', 470), ('парень', 447), ('бы', 440), ('

# Удаление частых слов и стоп слов

In [8]:
stopwords = nltk.corpus.stopwords.words('russian')

In [9]:
filter_significant = lambda s: ' '.join(
    list(filter(lambda w: not ((w, counts[w]) in most_common or (w, counts[w]) in stopwords), s.split()))
)

pd.DataFrame(anime['description'].apply(filter_significant)).to_csv('text_description.csv', index=False, header=False)

# Обучение модели для определения того аниме - фэнтези или нет

In [30]:
target = 'Фэнтези'

def is_target(r_g):
    return target in r_g

anime['isTarget'] = anime['genres'].apply(is_target)
anime

,name,score,aired_on,description,genres,isTarget
0,Ковбой Бибоп,8.75,1998-04-03,2071 год Человечество колонизировало всю Солне...,"['Экшен', 'Фантастика', 'Космос']",False
1,Легенда о новой Белоснежке Притиар,7.16,2001-04-04,Химэно Аваюки была обычной шестнадцатилетней...,"['Драма', 'Фэнтези', 'Романтика', 'Супер сила'...",True
2,Космический пират капитан Харлок,7.70,1978-03-14,2977 год Человечество перестало развиваться Вс...,"['Сэйнэн', 'Экшен', 'Приключения', 'Драма', 'Ф...",False
3,История из эпохи Сёва,6.84,2011-04-04,Драматическая история семьи Ямадзаки в 39м год...,"['Драма', 'Исторический']",False
4,Югио! Зексал,6.42,2011-04-11,История разворачивается в далёком будущем где ...,"['Экшен', 'Фэнтези', 'Сёнен']",True
...,...,...,...,...,...,...
2986,Уровень Е,7.41,2011-01-11,В наши дни на Земле проживают представители со...,"['Сёнен', 'Комедия', 'Фантастика']",False
2987,Скет Данс,8.21,2011-04-07,В академии Каймэй есть забавная троица клуб п...,"['Сёнен', 'Комедия', 'Школа']",False
2988,Синий экзорцист,7.49,2011-04-17,Когда заходит солнце наступает время демоновМ...,"['Сёнен', 'Экшен', 'Фэнтези', 'Школа']",True
2989,Гинтама 2,9.03,2011-04-04,Продолжение приключений Гинтоки и его команд...,"['Сёнен', 'Экшен', 'Комедия', 'Фантастика', 'И...",False


In [31]:
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import extract_unigram_feats
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

In [32]:
desc_train, desc_test, target_train, target_test = (
    train_test_split(anime['description'], anime['isTarget'], test_size=0.3))

In [33]:
classifier = SklearnClassifier(MLPClassifier(hidden_layer_sizes=(5,)))
analyzer = SentimentAnalyzer()

In [34]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import BpeTrainer

tokenizer = Tokenizer(BPE())
trainer = BpeTrainer(vocab_size=5000)
tokenizer.pre_tokenizer = Whitespace()
tokenizer.train(['text_description.csv'], trainer)

In [35]:
train_size = len(desc_train)
train_data = [None] * train_size
for i in range(train_size):
    train_data[i] = (
        tokenizer.encode(desc_train.iloc[i]).tokens,
        target_train.iloc[i]
    )

In [36]:
unigram_feats = analyzer.unigram_word_feats(analyzer.all_words(train_data))
analyzer.add_feat_extractor(extract_unigram_feats, unigrams=unigram_feats)
train_set = analyzer.apply_features(train_data, labeled=True)
analyzer.train(classifier.train, train_set)

Training classifier


<SklearnClassifier(MLPClassifier(hidden_layer_sizes=(5,)))>

In [37]:
from sklearn.metrics import accuracy_score

train_results = [None] * train_size
for i in range(train_size):
    train_results[i] = analyzer.classify(tokenizer.encode(desc_train.iloc[i]).tokens)

test_size = len(desc_test)
test_results = [None] * test_size
for i in range(test_size):
    test_results[i] = analyzer.classify(tokenizer.encode(desc_test.iloc[i]).tokens)
    
print("Train accuracy score: ", accuracy_score(target_train, train_results))
print("Test accuracy score: ", accuracy_score(target_test, test_results))

Train accuracy score:  1.0
Test accuracy score:  0.7895322939866369
